In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

# import xgboost as xgb
# import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.metrics import classification_report

from keras.models import load_model
from keras.utils import np_utils

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0", 
        allow_growth=True
    )
)

set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open('../data/HCD35_pos.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [3]:
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [4]:
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [5]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [6]:
d = {}

In [7]:
xgb_model = pickle.load(open('../model/xgb_HCD35_pos.sav', 'rb'))
xgb_result = xgb_model.predict_proba(X_test)
# print(xgb_model.score(X_test, y_test))
d['Xgboost'] = [round(xgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
rf_model = pickle.load(open('../model/rf_HCD35_pos.sav', 'rb'))
rf_result = rf_model.predict_proba(X_test)
print(rf_model.score(X_test, y_test))
d['Random Forest'] = [round(rf_model.score(X_test, y_test)*100, 2)]

0.7172619047619048


In [9]:
lgb_model = pickle.load(open('../model/lgbm_HCD35_pos.sav', 'rb'))
lgb_result = lgb_model.predict_proba(X_test)
# print(lgb_model.score(X_test, y_test))
d['LightGBM'] = [round(lgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
keras_model  = load_model('../model/Keras_HCD35_pos.h5')
keras_result = keras_model.predict(X_test)
d['Keras'] = [round(keras_model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2)]

In [11]:
o = {}

In [12]:
o['XGBoost + Ramdom Forest'] = [round((sum(np.argmax((xgb_result+rf_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM'] = [round((sum(np.argmax((xgb_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Keras'] = [round((sum(np.argmax((xgb_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['LightGBM+Keras'] = [round((sum(np.argmax((keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM + Keras'] = [round((sum(np.argmax((keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [13]:
b = pd.DataFrame()
b = pd.concat([b, pd.DataFrame(o).T], axis=1)

In [14]:
v = pd.DataFrame()
v = pd.concat([v, pd.DataFrame(d).T], axis=1)

In [15]:
with open('../data/HCD45_pos.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [16]:
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [17]:
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [18]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [19]:
d = {}

In [20]:
xgb_model = pickle.load(open('../model/xgb_HCD45_pos.sav', 'rb'))
xgb_result = xgb_model.predict_proba(X_test)
print(xgb_model.score(X_test, y_test))
d['Xgboost'] = [round(xgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7827380952380952


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
rf_model = pickle.load(open('../model/rf_HCD45_pos.sav', 'rb'))
rf_result = rf_model.predict_proba(X_test)
print(rf_model.score(X_test, y_test))
d['Random Forest'] = [round(rf_model.score(X_test, y_test)*100, 2)]

0.7083333333333334


In [22]:
lgb_model = pickle.load(open('../model/lgbm_HCD45_pos.sav', 'rb'))
lgb_result = lgb_model.predict_proba(X_test)
print(lgb_model.score(X_test, y_test))
d['LightGBM'] = [round(lgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8125


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
keras_model  = load_model('../model/Keras_HCD45_pos.h5')
keras_result = keras_model.predict(X_test)
d['Keras'] = [round(keras_model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2)]

In [24]:
o = {}

In [25]:
o['XGBoost + Ramdom Forest'] = [round((sum(np.argmax((xgb_result+rf_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM'] = [round((sum(np.argmax((xgb_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Keras'] = [round((sum(np.argmax((xgb_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['LightGBM+Keras'] = [round((sum(np.argmax((keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM + Keras'] = [round((sum(np.argmax((keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

In [26]:
b = pd.concat([b, pd.DataFrame(o).T], axis=1)

In [27]:
v = pd.concat([v, pd.DataFrame(d).T], axis=1)

In [28]:
with open('../data/HCD65_pos.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [29]:
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [30]:
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [31]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [32]:
d = {}

In [33]:
xgb_model = pickle.load(open('../model/xgb_HCD65_pos.sav', 'rb'))
xgb_result = xgb_model.predict_proba(X_test)
print(xgb_model.score(X_test, y_test))
d['Xgboost'] = [round(xgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7708333333333334


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [34]:
rf_model = pickle.load(open('../model/rf_HCD65_pos.sav', 'rb'))
rf_result = rf_model.predict_proba(X_test)
print(rf_model.score(X_test, y_test))
d['Random Forest'] = [round(rf_model.score(X_test, y_test)*100, 2)]

0.6517857142857143


In [35]:
lgb_model = pickle.load(open('../model/lgbm_HCD65_pos.sav', 'rb'))
lgb_result = lgb_model.predict_proba(X_test)
print(lgb_model.score(X_test, y_test))
d['LightGBM'] = [round(lgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8035714285714286


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
keras_model  = load_model('../model/Keras_HCD65_pos.h5')
keras_result = keras_model.predict(X_test)
d['Keras'] = [round(keras_model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2)]

In [37]:
o = {}

In [38]:
o['XGBoost + Ramdom Forest'] = [round((sum(np.argmax((xgb_result+rf_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM'] = [round((sum(np.argmax((xgb_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Keras'] = [round((sum(np.argmax((xgb_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['LightGBM+Keras'] = [round((sum(np.argmax((keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM + Keras'] = [round((sum(np.argmax((keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

In [39]:
b = pd.concat([b, pd.DataFrame(o).T], axis=1)

In [40]:
v = pd.concat([v, pd.DataFrame(d).T], axis=1)

In [41]:
with open('../data/HCD35_neg.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [42]:
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [43]:
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [44]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [45]:
d = {}

In [46]:
xgb_model = pickle.load(open('../model/xgb_HCD35_neg.sav', 'rb'))
xgb_result = xgb_model.predict_proba(X_test)
print(xgb_model.score(X_test, y_test))
d['Xgboost'] = [round(xgb_model.score(X_test, y_test)*100, 2)]

0.776595744680851


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [47]:
rf_model = pickle.load(open('../model/rf_HCD35_neg.sav', 'rb'))
rf_result = rf_model.predict_proba(X_test)
print(rf_model.score(X_test, y_test))
d['Random Forest'] = [round(rf_model.score(X_test, y_test)*100, 2)]

0.7127659574468085


In [48]:
lgb_model = pickle.load(open('../model/lgbm_HCD35_neg.sav', 'rb'))
lgb_result = lgb_model.predict_proba(X_test)
print(lgb_model.score(X_test, y_test))
d['LightGBM'] = [round(lgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7340425531914894


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [49]:
keras_model  = load_model('../model/Keras_HCD35_neg.h5')
keras_result = keras_model.predict(X_test)
d['Keras'] = [round(keras_model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2)]

In [50]:
o = {}

In [51]:
o['XGBoost + Ramdom Forest'] = [round((sum(np.argmax((xgb_result+rf_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM'] = [round((sum(np.argmax((xgb_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Keras'] = [round((sum(np.argmax((xgb_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['LightGBM+Keras'] = [round((sum(np.argmax((keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM + Keras'] = [round((sum(np.argmax((keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

In [52]:
b = pd.concat([b, pd.DataFrame(o).T], axis=1)

In [53]:
v = pd.concat([v, pd.DataFrame(d).T], axis=1)

In [54]:
with open('../data/HCD45_neg.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [55]:
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [56]:
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [57]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [58]:
d = {}

In [59]:
xgb_model = pickle.load(open('../model/xgb_HCD45_neg.sav', 'rb'))
xgb_result = xgb_model.predict_proba(X_test)
print(xgb_model.score(X_test, y_test))
d['Xgboost'] = [round(xgb_model.score(X_test, y_test)*100, 2)]

0.723404255319149


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [60]:
rf_model = pickle.load(open('../model/rf_HCD45_neg.sav', 'rb'))
rf_result = rf_model.predict_proba(X_test)
print(rf_model.score(X_test, y_test))
d['Random Forest'] = [round(rf_model.score(X_test, y_test)*100, 2)]

0.6702127659574468


In [61]:
lgb_model = pickle.load(open('../model/lgbm_HCD45_neg.sav', 'rb'))
lgb_result = lgb_model.predict_proba(X_test)
print(lgb_model.score(X_test, y_test))
d['LightGBM'] = [round(lgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7021276595744681


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [62]:
keras_model  = load_model('../model/Keras_HCD45_neg.h5')
keras_result = keras_model.predict(X_test)
d['Keras'] = [round(keras_model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2)]

In [63]:
o = {}

In [64]:
o['XGBoost + Ramdom Forest'] = [round((sum(np.argmax((xgb_result+rf_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM'] = [round((sum(np.argmax((xgb_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Keras'] = [round((sum(np.argmax((xgb_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['LightGBM+Keras'] = [round((sum(np.argmax((keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM + Keras'] = [round((sum(np.argmax((keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

In [65]:
b = pd.concat([b, pd.DataFrame(o).T], axis=1)

In [66]:
v = pd.concat([v, pd.DataFrame(d).T], axis=1)

In [67]:
with open('../data/HCD65_neg.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [68]:
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [69]:
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [70]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [71]:
d = {}

In [72]:
xgb_model = pickle.load(open('../model/xgb_HCD65_neg.sav', 'rb'))
xgb_result = xgb_model.predict_proba(X_test)
print(xgb_model.score(X_test, y_test))
d['Xgboost'] = [round(xgb_model.score(X_test, y_test)*100, 2)]

0.7446808510638298


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [73]:
rf_model = pickle.load(open('../model/rf_HCD65_neg.sav', 'rb'))
rf_result = rf_model.predict_proba(X_test)
print(rf_model.score(X_test, y_test))
d['Random Forest'] = [round(rf_model.score(X_test, y_test)*100, 2)]

0.648936170212766


In [74]:
lgb_model = pickle.load(open('../model/lgbm_HCD65_neg.sav', 'rb'))
lgb_result = lgb_model.predict_proba(X_test)
print(lgb_model.score(X_test, y_test))
d['LightGBM'] = [round(lgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6808510638297872


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [75]:
keras_model  = load_model('../model/Keras_HCD65_neg.h5')
keras_result = keras_model.predict(X_test)
d['Keras'] = [round(keras_model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2)]

In [76]:
o = {}

In [77]:
o['XGBoost + Ramdom Forest'] = [round((sum(np.argmax((xgb_result+rf_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM'] = [round((sum(np.argmax((xgb_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Keras'] = [round((sum(np.argmax((xgb_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['LightGBM+Keras'] = [round((sum(np.argmax((keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM + Keras'] = [round((sum(np.argmax((keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

In [78]:
b = pd.concat([b, pd.DataFrame(o).T], axis=1)
v = pd.concat([v, pd.DataFrame(d).T], axis=1)
s = pd.concat([b, v])
s.columns = [
    'HCD35_Positive', 'HCD45_Positive', 'HCD65_Positive',
    'HCD35_Negative', 'HCD45_Negative', 'HCD65_Negative',
]

In [79]:
s.to_csv('../result/voting.csv')